# Приготовления

Этот colab notebook является минимальной демонстрацией для Faceswap GAN.
Поскольку colab допускает максимальное время работы 12 часов, мы будем
обучать только облегченную модель в этом notebook.

**Цель этой notebook - не обучить модель, которая дает высококачественные результаты, а дать краткий обзор того, как работает faceswap-GAN.**

Порядок работы faceswap-GAN описан ниже:

  1. Загрузите два видео для обучения;
  2. Примените извлечение лица (предварительную обработку) к двум загруженным видео;
  3. Тренируйте легкую модель для Faceswap GAN. (~10-12 ч.)
  4. Примените преобразование видео к загруженным видео.

# Шаг 1: Установите тип среды выполнения на Python 3/GPU
Установите ноутбук colab в экземпляр GPU с помощью:
**runtime -> change runtime type -> Python3 and GPU**

В следующих ячейках будет отображена системная информация текущего экземпляра.
Запустите ячейки и проверьте, использует ли он python > = 3.6 и имеет ли устройство GPU.

In [ ]:
import platform

print(platform.python_version())

Установите зависимости

In [ ]:
!wget https://gist.githubusercontent.com/alvinahmadov/ff20d421cc9e9798564ed1628ad90d66/raw/e5b0890c33f6f27ccb6432c5274edb06e9bccd1d/requirements.txt

%pip install -r requirements.txt

In [ ]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

# Step 2: Клонируйте репозиторий

In [ ]:
!git clone https://github.com/alvinahmadov/faceswap-parts.git

In [ ]:
%cd "faceswap-parts"

# Шаг 3: Загрузите видеоролики обучения

Пользователь должен загрузить два видео: **source video** и **target video**.
Модель **преобразует исходное лицо в целевое по умолчанию.**

  - Видео, для лучшего результата должен **содержать только одного человека**.
  - Ограничений по длине видео нет, но чем оно длиннее, тем больше времени потребуется
на предварительную обработку/преобразование видео, что может привести к увеличению времени
выполнения до 12 часов. (**Рекомендуемая продолжительность видео: 30 секунд ~ 2 минуты.**)

In [ ]:
try:
    # noinspection PyUnresolvedReferences,PyPackageRequirements
    from google.colab import files
except:
    print("This notebook can be run only in google colab")
    pass

In [ ]:
# Upload source video
source_video = files.upload()

for fn_source_video, _ in source_video.items():
    print(fn_source_video)
    pass

In [ ]:
# Upload target video
target_video = files.upload()

for fn_target_video, _ in target_video.items():
    print(fn_target_video)
    pass

# Шаг 4: Установите максимальное количество итераций обучения
Для 25000 итераций по умолчанию требуется ~10 часов обучения.

Итерации >= 27k могут превышать предельное время выполнения;
Итерации < 18k могут привести к плохо обученной модели.

In [ ]:
TOTAL_ITERS = 20000

# Шаг 5: Все готово.

**Нажмите Ctrl + F10 (или runtime -> run after)**, чтобы запустить процесс. Для завершения
тренировки потребуется 10 ~ 12 часов. Результирующее видео можно загрузить, запустив
последнюю ячейку:
  ```shell
  files.download("OUTPUT_VIDEO.mp4")
  ```
Обратите внимание, что **эту страницу не следует закрывать или обновлять во время работы**.

In [ ]:
%%capture
import imageio

# noinspection PyUnresolvedReferences
imageio.plugins.ffmpeg.download()

In [ ]:
import keras.backend as K
from detector.face_detector import MTCNNFaceDetector
import glob

from preprocess import preprocess_video

In [ ]:
fd = MTCNNFaceDetector(sess=K.get_session(), model_path="./mtcnn_weights/")

In [ ]:
!mkdir -p face_src/rgb
!mkdir -p face_src/binary_mask
!mkdir -p face_dst/rgb
!mkdir -p face_dst/binary_mask

In [ ]:
save_interval = 5 # perform face detection every {save_interval} frames
save_path = "./face_src/"
# noinspection PyUnboundLocalVariable
preprocess_video(fn_source_video, fd, save_interval, save_path)
save_path = "./face_dst/"
# noinspection PyUnboundLocalVariable
preprocess_video(fn_target_video, fd, save_interval, save_path)

In [ ]:
face_src_glob_len=str(len(glob.glob("face_src/rgb/*.*")))
face_dst_glob_len=str(len(glob.glob("face_dst/rgb/*.*")))

print(f"{face_src_glob_len} face(s) extracted from source video: {fn_source_video}.")
print(f"{face_dst_glob_len} face(s) extracted from target video: {fn_target_video}.")

## Следующие ячейки взяты из [faceswap_train_test.ipynb](https://github.com/alvinahmadov/faceswap-parts/blob/main/faceswap_train_test.ipynb)

## Импортируйте пакеты

In [ ]:
import keras.backend as K
import tensorflow as tf

In [ ]:
import os
import glob
import time
from pathlib import Path
from IPython.display import clear_output

%matplotlib inline

## Конфигурация

In [ ]:
K.set_learning_phase(1)
# Number of CPU cores
num_cpus = os.cpu_count()

# Input/Output resolution
RESOLUTION = 64  # 64x64, 128x128, 256x256
assert (RESOLUTION % 64) == 0, "RESOLUTION should be 64, 128, or 256."

batch_size = 4

# Use motion blur (data augmentation)
# set True if training data contains images extracted from videos
use_da_motion_blur = False

# Use eye-aware training
# require images generated from prep_binary_masks.ipynb
use_bm_eyes = True

# Probability of random color matching (data augmentation)
prob_random_color_match = 0.5

da_config = {
    "prob_random_color_match": prob_random_color_match,
    "use_da_motion_blur": use_da_motion_blur,
    "use_bm_eyes": use_bm_eyes
}

# Path to training images
img_dir_src = './face_src/rgb' # source face
img_dir_dst = './face_dst/rgb' # target face
img_dir_src_bm_eyes = "./face_src/binary_mask"
img_dir_dst_bm_eyes = "./face_dst/binary_mask"

# Path to saved model weights
models_dir = "./models"

# Architecture configuration
arch_config = {
    "IMAGE_SHAPE": (RESOLUTION, RESOLUTION, 3),
    "use_self_attn": True,
    "norm": "hybrid",
    "model_capacity": "lite"
}

# Loss function weights configuration
loss_weights = {
    "w_D": 0.1,
    "w_recon": 1.,
    "w_edge": 0.1,
    "w_eyes": 30.,
    "w_pl": (0.01, 0.1, 0.3, 0.1)
}

# Init. loss config.
loss_config = {
    "gan_training": "mixup_LSGAN",
    "use_PL": False,
    "PL_before_activ": True,
    "use_mask_hinge_loss": False,
    "m_mask": 0.,
    "lr_factor": 1.,
    "use_cyclic_loss": False
}

## Постройте модель

In [ ]:
from networks.faceswap_model import FaceswapModel
from data_loader import DataLoader
from utils import showG, showG_mask

In [ ]:
model: FaceswapModel = FaceswapModel(**arch_config)

In [ ]:
%%capture
!wget https://github.com/rcmalli/keras-vggface/releases/download/v2.0/rcmalli_vggface_tf_notop_resnet50.h5

In [ ]:
from vggface_models import RESNET50

vggface = RESNET50(include_top=False, weights=None, input_shape=(224, 224, 3))
vggface.load_weights("rcmalli_vggface_tf_notop_resnet50.h5")

model.build_pl_model(vggface_model=vggface, before_activ=loss_config["PL_before_activ"])
model.build_train_functions(loss_weights=loss_weights, **loss_config)

## Начните тренировку

In [ ]:
# Create ./models directory
Path(f"models").mkdir(parents=True, exist_ok=True)

In [ ]:
# Get file names
train_src = glob.glob(f"{img_dir_src}/*.*")
train_dst = glob.glob(f"{img_dir_dst}/*.*")

train_src_n_dst = train_src + train_dst

assert len(train_src), f"Изображение не найдено в {img_dir_src}"
assert len(train_dst), f"Изображение не найдено в {img_dir_dst}"
print(f"Количество изображений в папке A: {str(len(train_src))}")
print(f"Количество изображений в папке B: {str(len(train_dst))}")

In [ ]:
def show_loss_config(loss_conf):
    for config, value in loss_conf.items():
        print(f"{config} = {value}")
        pass
    pass

In [ ]:
def reset_session(spath):
    """
    Parameters
    ----------
    spath : str
     Save path
    """
    global model, vggface
    global train_batch_src, train_batch_dst
    model.save_weights(path=spath)
    del model
    del vggface
    del train_batch_src
    del train_batch_dst
    K.clear_session()
    model = FaceswapModel(**arch_config)
    model.load_weights(path=spath)
    vggface = RESNET50(include_top=False, weights=None, input_shape=(224, 224, 3))
    vggface.load_weights("rcmalli_vggface_tf_notop_resnet50.h5")
    model.build_pl_model(vggface_model=vggface, before_activ=loss_config["PL_before_activ"])
    train_batch_src = DataLoader(filenames=train_src, all_filenames=train_src_n_dst,
                                 batch_size=batch_size, dir_bm_eyes=img_dir_src_bm_eyes,
                                 resolution=RESOLUTION, num_cpus=num_cpus, session=K.get_session(),
                                 **da_config)
    train_batch_dst = DataLoader(filenames=train_dst, all_filenames=train_src_n_dst,
                                 batch_size=batch_size, dir_bm_eyes=img_dir_dst_bm_eyes,
                                 resolution=RESOLUTION, num_cpus=num_cpus, session=K.get_session(),
                                 **da_config)
    pass

In [ ]:
# Start training
t0 = time.time()

# This try/except is meant to resume training if we disconnected from Colab
try:
    # noinspection PyUnresolvedReferences,PyStatementEffect,PyUnboundLocalVariable
    gen_iterations
    # noinspection PyUnresolvedReferences,PyStatementEffect,PyUnboundLocalVariable
    print(f"Возобновить обучение c {gen_iterations} итерации.")
except:
    gen_iterations = 0
    pass

errGA_sum = errGB_sum = errDA_sum = errDB_sum = 0
errGAs = {}
errGBs = {}

for k in ['ttl', 'adv', 'recon', 'edge', 'pl']:
    errGAs[k] = 0
    errGBs[k] = 0
    pass

display_iters = 300
global TOTAL_ITERS
global train_batch_src, train_batch_dst

train_batch_src = DataLoader(train_src, train_src_n_dst, batch_size,
                             dir_bm_eyes=img_dir_src_bm_eyes, resolution=RESOLUTION,
                             num_cpus=num_cpus, session=K.get_session(), **da_config)

train_batch_dst = DataLoader(train_dst, train_src_n_dst, batch_size,
                             dir_bm_eyes=img_dir_dst_bm_eyes, resolution=RESOLUTION,
                             num_cpus=num_cpus, session=K.get_session(), **da_config)

while gen_iterations <= TOTAL_ITERS:
    # Loss function automation
    if gen_iterations == (TOTAL_ITERS // 5 - display_iters // 2):
        clear_output()
        loss_config['use_PL'] = True
        loss_config['use_mask_hinge_loss'] = False
        loss_config['m_mask'] = 0.0
        reset_session(models_dir)
        print("Конструкция новых функций потерь...")
        show_loss_config(loss_config)
        model.build_train_functions(loss_weights=loss_weights, **loss_config)
        print("Выполнено.")
        pass
    elif gen_iterations == (TOTAL_ITERS // 5 + TOTAL_ITERS // 10 - display_iters // 2):
        clear_output()
        loss_config['use_PL'] = True
        loss_config['use_mask_hinge_loss'] = True
        loss_config['m_mask'] = 0.5
        reset_session(models_dir)
        print("Конструкция новых функций потерь...")
        show_loss_config(loss_config)
        model.build_train_functions(loss_weights=loss_weights, **loss_config)
        print("Завершено.")
        pass
    elif gen_iterations == (2 * TOTAL_ITERS // 5 - display_iters // 2):
        clear_output()
        loss_config['use_PL'] = True
        loss_config['use_mask_hinge_loss'] = True
        loss_config['m_mask'] = 0.2
        reset_session(models_dir)
        print("Конструкция новых функций потерь...")
        show_loss_config(loss_config)
        model.build_train_functions(loss_weights=loss_weights, **loss_config)
        print("Выполнено.")
        pass
    elif gen_iterations == (TOTAL_ITERS // 2 - display_iters // 2):
        clear_output()
        loss_config['use_PL'] = True
        loss_config['use_mask_hinge_loss'] = True
        loss_config['m_mask'] = 0.4
        loss_config['lr_factor'] = 0.3
        reset_session(models_dir)
        print("Конструкция новых функций потерь...")
        show_loss_config(loss_config)
        model.build_train_functions(loss_weights=loss_weights, **loss_config)
        print("Выполнено.")
        pass
    elif gen_iterations == (2 * TOTAL_ITERS // 3 - display_iters // 2):
        clear_output()
        model.decoder_src.load_weights("models/decoder_B.h5")  # swap decoders
        model.decoder_dst.load_weights("models/decoder_A.h5")  # swap decoders
        loss_config['use_PL'] = True
        loss_config['use_mask_hinge_loss'] = True
        loss_config['m_mask'] = 0.5
        loss_config['lr_factor'] = 1
        reset_session(models_dir)
        print("Конструкция новых функций потерь...")
        show_loss_config(loss_config)
        model.build_train_functions(loss_weights=loss_weights, **loss_config)
        print("Выполнено.")
        pass
    elif gen_iterations == (8 * TOTAL_ITERS // 10 - display_iters // 2):
        clear_output()
        loss_config['use_PL'] = True
        loss_config['use_mask_hinge_loss'] = True
        loss_config['m_mask'] = 0.1
        loss_config['lr_factor'] = 0.3
        reset_session(models_dir)
        print("Конструкция новых функций потерь...")
        show_loss_config(loss_config)
        model.build_train_functions(loss_weights=loss_weights, **loss_config)
        print("Выполнено.")
        pass
    elif gen_iterations == (9 * TOTAL_ITERS // 10 - display_iters // 2):
        clear_output()
        loss_config['use_PL'] = True
        loss_config['use_mask_hinge_loss'] = False
        loss_config['m_mask'] = 0.0
        loss_config['lr_factor'] = 0.1
        reset_session(models_dir)
        print("Конструкция новых функций потерь...")
        show_loss_config(loss_config)
        model.build_train_functions(loss_weights=loss_weights, **loss_config)
        print("Выполнено.")

        pass

    if gen_iterations == 5:
        print("working.")
        pass

    # Train dicriminators for one batch
    data_src = train_batch_src.get_next_batch()
    data_dst = train_batch_dst.get_next_batch()
    errDA, errDB = model.train_one_batch_disc(data_src, data_dst)
    errDA_sum += errDA[0]
    errDB_sum += errDB[0]

    # Train generators for one batch
    data_src = train_batch_src.get_next_batch()
    data_dst = train_batch_dst.get_next_batch()
    errGA, errGB = model.train_one_batch_gen(data_src, data_dst)
    errGA_sum += errGA[0]
    errGB_sum += errGB[0]
    for i, k in enumerate(['ttl', 'adv', 'recon', 'edge', 'pl']):
        errGAs[k] += errGA[i]
        errGBs[k] += errGB[i]
        pass
    gen_iterations += 1

    # Visualization
    if gen_iterations % display_iters == 0:
        clear_output()

        # Display loss information
        show_loss_config(loss_config)
        print("----------")
        print("[iter %d] Loss_DA: %f Loss_DB: %f Loss_GA: %f Loss_GB: %f time: %f"
              % (gen_iterations, errDA_sum / display_iters, errDB_sum / display_iters,
                 errGA_sum / display_iters, errGB_sum / display_iters, time.time() - t0))
        print("----------")
        print("Детали потерь генератора:")
        print(f"[Adversarial loss]\nGA: {errGAs['adv'] / display_iters:.4f} GB: {errGBs['adv'] / display_iters:.4f}")
        print(f"[Reconstruction loss]\nGA: {errGAs['recon'] / display_iters:.4f} GB: {errGBs['recon'] / display_iters:.4f}")
        print(f"[Edge loss]\nGA: {errGAs['edge'] / display_iters:.4f} GB: {errGBs['edge'] / display_iters:.4f}")
        if loss_config['use_PL']:
            print(f"[Perceptual loss]")
            try:
                print(f"GA: {errGAs['pl'][0] / display_iters:.4f} GB: {errGBs['pl'][0] / display_iters:.4f}")
            except:
                print(f"GA: {errGAs['pl'] / display_iters:.4f} GB: {errGBs['pl'] / display_iters:.4f}")
                pass
            pass

        # Display images
        print("----------")
        w_src, t_src, _ = train_batch_src.get_next_batch()
        w_dst, t_dst, _ = train_batch_dst.get_next_batch()
        print("Преобразованные (замаскированные) результаты:")
        showG(t_src, t_dst, model.path_src, model.path_dst, batch_size)
        print("Маски:")
        showG_mask(t_src, t_dst, model.path_mask_src, model.path_mask_dst, batch_size)
        print("Результаты реконструкции:")
        showG(w_src, w_dst, model.path_bgr_src, model.path_bgr_dst, batch_size)
        errGA_sum = errGB_sum = errDA_sum = errDB_sum = 0
        for k in ['ttl', 'adv', 'recon', 'edge', 'pl']:
            errGAs[k] = 0
            errGBs[k] = 0
            pass

        # Save models
        model.save_weights(path=models_dir)
        pass
    pass

## Конвертация видео
[faceswap_video_conversion.ipynb](https://github.com/alvinahmadov/faceswap-parts/blob/main/colab/faceswap_video_conversion.ipynb)

In [ ]:
from converter import VideoConverter

In [ ]:
global model, vggface
global train_batch_src, train_batch_b
del model
del vggface
del train_batch_src
del train_batch_dst
tf.compat.v1.reset_default_graph()
K.clear_session()
model = FaceswapModel(**arch_config)
model.load_weights(path=models_dir)

In [ ]:
fd = MTCNNFaceDetector(sess=K.get_session(), model_path="./mtcnn_weights/")
vc = VideoConverter()
vc.set_face_detector(fd)
vc.set_gan_model(model)

In [ ]:
options = {
    # ===== Fixed =====
    "use_smoothed_bbox": True,
    "use_kalman_filter": True,
    "use_auto_downscaling": False,
    "bbox_moving_avg_coef": 0.65,
    "min_face_area": 35 * 35,
    "IMAGE_SHAPE": model.image_shape,
    # ===== Tunable =====
    "kf_noise_coef": 1e-3,
    "use_color_correction": "hist_match",
    "detec_threshold": 0.8,
    "roi_coverage": 0.9,
    "enhance": 0.,
    "output_type": 3,
    "direction": "AtoB",  # determines the transform direction
}

In [ ]:
if options["direction"] == "AtoB":
    input_fn = fn_source_video
    output_fn = "OUTPUT_VIDEO_AtoB.mp4"
    pass
elif options["direction"] == "BtoA":
    input_fn = fn_target_video
    output_fn = "OUTPUT_VIDEO_BtoA.mp4"
    pass

duration = None  # None or a non-negative float tuple: (start_sec, end_sec). Duration of input video to be converted

In [ ]:
# noinspection PyUnboundLocalVariable
vc.convert(input_fn=input_fn, output_fn=output_fn, options=options, duration=duration)

# Скачать результат (видеофайл)

In [ ]:
try:
    # noinspection PyUnresolvedReferences,PyPackageRequirements
    from google.colab import files
except:
    print("This notebook can be run only in google colab")
    pass

In [ ]:
if options["direction"] == "AtoB":
    files.download("OUTPUT_VIDEO_AtoB.mp4")
    pass
elif options["direction"] == "BtoA":
    files.download("OUTPUT_VIDEO_BtoA.mp4")
    pass